In [1]:
''' 1 января 2016 года по 1 июля 2019 года
30177909
В рамках второй гипотезы нам нужно оценить количество роботов, которые заходят на наш сайт. 
То есть: определить с каких источников и в какое время суток чаще всего заходят роботы. 
Роботами могут быть пауки поисковых систем, которые индексируют контент блога, или же различные парсеры и агрегаторы контента. 
Яндекс.Метрика позволяет находить этих роботов и показывает их в отдельном параметрe (ym:s:isRobot). 
Эту гипотезу вам предстоит проверить самостоятельно.'''

' 1 января 2016 года по 1 июля 2019 года\n30177909\nВ рамках второй гипотезы нам нужно оценить количество роботов, которые заходят на наш сайт. \nТо есть: определить с каких источников и в какое время суток чаще всего заходят роботы. \nРоботами могут быть пауки поисковых систем, которые индексируют контент блога, или же различные парсеры и агрегаторы контента. \nЯндекс.Метрика позволяет находить этих роботов и показывает их в отдельном параметрe (ym:s:isRobot). \nЭту гипотезу вам предстоит проверить самостоятельно.'

In [2]:
import yaml
from pathlib2 import Path
import requests
import json
import pandas as pd

from pprint import pprint

my_token = '1234567890' #  тут надо указать свой yandex токен
try:
    pathstring = '/Users/phaggi/Documents/python/PythonNotes/analit/security.yml'
    file = Path(pathstring)
    with open(file, 'r') as input_file:
        security = yaml.safe_load(input_file)
        ya_token = security['ya_token']
except:
    ya_token = my_token

date_from = '2016-01-01'
date_to = '2019-07-01'
ids = 30177909

params = {'metrics': 'ym:s:visits', 
          'dimensions': 'ym:s:hour,ym:s:isRobot,ym:s:lastsignTrafficSource',
          'date1': date_from,  
          'date2': date_to, 
          'limit': 100000,
          'accuracy':'full',
          'ids': ids}
headers = {'Authorization': 'OAuth ' + ya_token}

url = "https://api-metrika.yandex.net/stat/v1/data"
response = requests.get(url, params=params, headers=headers)

In [3]:
metrika_data = response.json()
# Создаём пустой список, в который будем добавлять словари в новом формате
result = []

# Начинаем перебор элементов старого отчёта (только раздел с данными)
for data_item in metrika_data['data']:
    # Создаём словарь для хранения текущих данных в новом формате 
    new_dict = {}  
    # Обращаемся к разделу отчёта, содержащего метаданные
    # Перебираем названия группировок, которые использовались в отчёте
    for i,dimension in enumerate(data_item['dimensions']):
        # Создаём в новом словаре ключ для каждой группировки
        # Устанавливаем значение для каждого ключа
        new_dict[metrika_data['query']['dimensions'][i]] = dimension['name']
    # Те же действия выполняем для метрик
    for i,metric in enumerate(data_item['metrics']):
        new_dict[metrika_data['query']['metrics'][i]] = metric
    # Добавляем созданный словарь в итоговый список result
    result.append(new_dict)

df = pd.read_json(json.dumps(result))

In [4]:
df['ym:s:visits'] = df['ym:s:visits'].astype('int')
df = df.dropna(subset=['ym:s:lastsignTrafficSource'])
df.columns = [col.replace('ym:s:','') for col in df.columns]
df = df[df['isRobot'] == 'Robots']
robots_hours = df.groupby('hour')['visits'].sum().idxmax()
robots_hours_maxvalue = df.groupby('hour')['visits'].sum().max()
robots_source = df.groupby('lastsignTrafficSource')['visits'].sum().idxmax()
robots_source_max = df.groupby('lastsignTrafficSource')['visits'].sum().max()
print('Ответ: чаще всего заходят роботы с {} ({} визитов), по времени чаще в {} часов ({} визитов)'\
      .format(robots_source, robots_source_max, robots_hours, robots_hours_maxvalue))

Ответ: чаще всего заходят роботы с Direct traffic (1826 визитов), по времени чаще в 11:00 часов (179 визитов)
